In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from dotenv import load_dotenv

load_dotenv()

True

In [4]:

doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )


In [5]:
docs = [doc1, doc2, doc3, doc4, doc5]

In [6]:
vector_store = Chroma(
    embedding_function=GoogleGenerativeAIEmbeddings(model="gemini-embedding-001"),
    persist_directory="./chroma_db",
    collection_name="sample"
)

In [7]:
# Add
vector_store.add_documents(
    documents=docs
)

['ab6d77db-66c8-463d-bdd7-ad0ce2edd574',
 '16933517-cea4-42e7-9e81-d218a037a934',
 '6df996b1-77f0-4c3f-9628-cb93b562fb3a',
 'ffd1aa4e-182f-4092-bcf7-257a64e6d0c8',
 'c4e64484-4611-44b1-b0ef-48fb0aadd5e0']

In [8]:
# Retrive
vector_store.get(
    include=["embeddings", "documents", "metadatas"]
)

{'ids': ['ab6d77db-66c8-463d-bdd7-ad0ce2edd574',
  '16933517-cea4-42e7-9e81-d218a037a934',
  '6df996b1-77f0-4c3f-9628-cb93b562fb3a',
  'ffd1aa4e-182f-4092-bcf7-257a64e6d0c8',
  'c4e64484-4611-44b1-b0ef-48fb0aadd5e0'],
 'embeddings': array([[-0.00982054,  0.02545763,  0.02402782, ...,  0.01414876,
         -0.01560954, -0.00266117],
        [-0.01989721,  0.01092714,  0.01730603, ...,  0.00973945,
         -0.0176257 , -0.00460104],
        [-0.01358705, -0.00359449,  0.01163961, ...,  0.01149882,
         -0.02098301,  0.00373549],
        [-0.01261678, -0.00227585,  0.00470995, ..., -0.00144414,
          0.00646786, -0.00529741],
        [-0.01413488, -0.02750698,  0.01302837, ...,  0.00987475,
         -0.00950909, -0.00272136]], shape=(5, 3072)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the m

In [9]:
# Similarity Search
vector_store.similarity_search(
    query="Who among these are caption ?", 
    k=2
)

[Document(id='6df996b1-77f0-4c3f-9628-cb93b562fb3a', metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
 Document(id='16933517-cea4-42e7-9e81-d218a037a934', metadata={'team': 'Mumbai Indians'}, page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.")]

In [10]:
res = vector_store.similarity_search(
    query="Give me any bowler name ?", 
    k=1

)
print(res[0].page_content)

Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.


In [15]:
# meta-data filtering
vector_store.similarity_search_with_score(
    query=" ",
    filter={"team": "Chennai Super Kings"}
)

[(Document(id='6df996b1-77f0-4c3f-9628-cb93b562fb3a', metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  0.5289300084114075),
 (Document(id='c4e64484-4611-44b1-b0ef-48fb0aadd5e0', metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  0.5591708421707153)]

In [16]:
update_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

In [19]:
# update doc
vector_store.update_document('ab6d77db-66c8-463d-bdd7-ad0ce2edd574', document=update_doc1)

In [20]:
vector_store.get(ids='ab6d77db-66c8-463d-bdd7-ad0ce2edd574')

{'ids': ['ab6d77db-66c8-463d-bdd7-ad0ce2edd574'],
 'embeddings': None,
 'documents': ["Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket."],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'team': 'Royal Challengers Bangalore'}]}

In [21]:
# delete
vector_store.delete(ids='ab6d77db-66c8-463d-bdd7-ad0ce2edd574')

In [22]:
# Apply where condition in vector_store.get()
vector_store.get(
    where={"team": "Mumbai Indians"},
    include=["documents", "metadatas"]
)

{'ids': ['16933517-cea4-42e7-9e81-d218a037a934',
  'ffd1aa4e-182f-4092-bcf7-257a64e6d0c8'],
 'embeddings': None,
 'documents': ["Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
  'Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'],
 'uris': None,
 'included': ['documents', 'metadatas'],
 'data': None,
 'metadatas': [{'team': 'Mumbai Indians'}, {'team': 'Mumbai Indians'}]}

In [23]:
vector_store.get()

{'ids': ['16933517-cea4-42e7-9e81-d218a037a934',
  '6df996b1-77f0-4c3f-9628-cb93b562fb3a',
  'ffd1aa4e-182f-4092-bcf7-257a64e6d0c8',
  'c4e64484-4611-44b1-b0ef-48fb0aadd5e0'],
 'embeddings': None,
 'documents': ["Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
  'MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.',
  'Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.',
  'Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas':